# Plot EOS models

In [ ]:
from cmcrameri import cm
import logging
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import numpy as np

from atmodeller import debug_logger
from atmodeller.eos.interfaces import RealGas
from atmodeller.eos.holland import get_holland_eos_models
from atmodeller.eos.holley import get_holley_eos_models
from atmodeller.eos.saxena import get_saxena_eos_models

In [ ]:
logger = debug_logger()
logger.setLevel(logging.INFO)

In [ ]:
matplotlib.rcParams.update({'font.size': 20})

In [ ]:
def plot_eos_collection(eos_models: dict[str, RealGas], pressure_min: float=0.1, 
        pressure_max: float=950, temperature_min: float=300, temperature_max: float=1000, 
        delta_temperature: float=100, xmin: float=0, xmax: float=1000, xscale:str='linear', 
        ymin: float=0, ymax: float=2, yscale: str='linear', 
        suptitle: str="Fugacity coefficients") -> Figure:
    """Plots the fugacity coefficients from a collection of EOS
    
    Args:
        eos_collection: A collection of EOS
        pressure_min: Minimum pressure in bar. Defaults to 0.1 bar.
        pressure_max: Maximum pressure in bar. Defaults to 950 bar.
        temperature_min: Minimum temperature in K. Defaults to 300 K.
        temperature_max: Maximum temperature in K. Defaults to 1000 K.
        delta_temperature: Temperature difference in K. Defaults to 100 K.
        xmin: Minimum pressure for x-axis. Defaults to 0 bar.
        xmax: Maximum pressure for x-axis. Defaults to 1000 bar.
        xscale: Scale for the x-axis. Defaults to linear.
        ymin: Minimum fugacity coefficient for y-axis. Defaults to 0.
        ymax: Maximium fugacity coefficient for y-axis. Defaults to 2.
        yscale: Scale for the y-axis. Defaults to linear.
        suptitle: Super title. Defaults to Fugacity coefficients.

    Returns:
        Figure handle
    """
    # Species in alphabetical order
    eos_species = sorted(list(eos_models.keys()))

    # These bounds are to ensure the EOS is plotted within its roughly calibrated region.
    pressure = np.linspace(pressure_min, pressure_max, 100)
    temperatures = np.arange(temperature_min, temperature_max+1, delta_temperature)

    colormap = cm.lajolla # type: ignore
    colors = colormap((temperatures-temperatures[0])/(temperatures[-1]-temperatures[0]))
    colormap_dict = {temperature: color for temperature, color in zip(temperatures, colors)}

    # 2 cms less in each dimension compared to A4 portrait
    fig, axs = plt.subplots(nrows=6, ncols=2, figsize=(19, 27.7))

    for nn, ax in enumerate(axs.reshape(-1)):
        try:
            species_name = eos_species[nn]
        except IndexError:
            ax.set_axis_off()
            continue
        eos = eos_models[species_name]
        logger.info("Plotting %s", species_name)
        for temperature in temperatures:
            temperature_array = temperature * np.ones_like(pressure)
            eos.calibration.check(temperature, pressure.min(), species_name)
            eos.calibration.check(temperature, pressure.max(), species_name)
            fugacity_coefficient = np.vectorize(eos.fugacity_coefficient)
            coefficient = fugacity_coefficient(temperature_array, pressure)
            ax.plot(pressure, coefficient, label=f"{temperature} K", 
                    color=colormap_dict[temperature])
            ax.set_title(species_name)
            ax.set_xlim((xmin, xmax))
            ax.set_ylim((ymin, ymax))

        ax.set_xlabel(r"Pressure (bar)")
        ax.set_ylabel(r'Fugacity Coefficient ($\phi = f/P$)')
        ax.set_xscale(xscale)
        ax.set_yscale(yscale)
        ax.legend(ncols=2)

    fig.suptitle(suptitle)
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)

    return fig

In [ ]:
holley_eos = get_holley_eos_models()
holland_eos = get_holland_eos_models()
saxena_eos = get_saxena_eos_models()

## Holley models

In [ ]:
plot_eos_collection(holley_eos, suptitle="Beattie-Bridgeman EOS (Holley et al., 1958)");

## Holland models

In [ ]:
plot_eos_collection(holland_eos, suptitle="CORK EOS (Holland and Powell) low P-T");

In [ ]:
plot_eos_collection(holland_eos, pressure_max=5E4, temperature_min=1000, temperature_max=3000, delta_temperature=500,
    xmax=5E4, ymin=0.1, ymax=1E4, yscale='log', suptitle="CORK EOS (Holland and Powell) high P-T");

## Saxena models

In [ ]:
plot_eos_collection(saxena_eos, suptitle="Corresponding states (Saxena et al.) low P-T");

In [ ]:
plot_eos_collection(saxena_eos, pressure_min=100, pressure_max=5E4, temperature_min=1000, temperature_max=3000, delta_temperature=500,
    xmax=5E4, ymin=0.1, ymax=1E4, yscale='log', suptitle="Corresponding states (Saxena et al.) high P-T");